In [16]:
import pandas as pd

from thresh import load_interface
import git, os, shutil, getpass
from github import Github

In [8]:
df = pd.read_csv("/home/li.mil/hierarchical-mds/human_eval/csv/preference_ranking.csv")
df

,Unnamed: 0,label,output
0,0,"At present, there is insufficient RCT evidence...",Output 1: The evidence from the provided snipp...
1,1,There is inadequate evidence to draw strong co...,Output 1: The evidence suggests that various i...
2,2,Collaborative care is associated with signific...,Output 1: The evidence suggests that collabora...
3,3,Administration of systemic prophylactic antibi...,Output 1: The evidence from the provided artic...
4,4,The limited available evidence suggests folate...,Output 1: The evidence from the provided snipp...
...,...,...,...
465,465,There was evidence from three pooled RCTs that...,Output 1: The evidence from several studies su...
466,466,Antibiotic prophylaxis is effective in prevent...,Output 1: The evidence from the provided artic...
467,467,The practice of surgical and chemical sympathe...,Output 1: The evidence suggests that sympathec...
468,468,There is not sufficient evidence to support or...,Output 1: The evidence from the provided snipp...


In [9]:
df.rename(columns={'label':'source', 'output': 'target'}, inplace=True)

In [10]:
df

,Unnamed: 0,source,target
0,0,"At present, there is insufficient RCT evidence...",Output 1: The evidence from the provided snipp...
1,1,There is inadequate evidence to draw strong co...,Output 1: The evidence suggests that various i...
2,2,Collaborative care is associated with signific...,Output 1: The evidence suggests that collabora...
3,3,Administration of systemic prophylactic antibi...,Output 1: The evidence from the provided artic...
4,4,The limited available evidence suggests folate...,Output 1: The evidence from the provided snipp...
...,...,...,...
465,465,There was evidence from three pooled RCTs that...,Output 1: The evidence from several studies su...
466,466,Antibiotic prophylaxis is effective in prevent...,Output 1: The evidence from the provided artic...
467,467,The practice of surgical and chemical sympathe...,Output 1: The evidence suggests that sympathec...
468,468,There is not sufficient evidence to support or...,Output 1: The evidence from the provided snipp...


In [11]:
df.head()

,Unnamed: 0,source,target
0,0,"At present, there is insufficient RCT evidence...",Output 1: The evidence from the provided snipp...
1,1,There is inadequate evidence to draw strong co...,Output 1: The evidence suggests that various i...
2,2,Collaborative care is associated with signific...,Output 1: The evidence suggests that collabora...
3,3,Administration of systemic prophylactic antibi...,Output 1: The evidence from the provided artic...
4,4,The limited available evidence suggests folate...,Output 1: The evidence from the provided snipp...


In [12]:
df.sample(10).to_json("cochrane.json", orient='records')

In [13]:
interface = load_interface('interface.yml')

In [14]:
local_template_path = "interface.yml"
local_data_path = "cochrane.json"
repository_name = "prolific-deployment-ml"

In [25]:
if os.path.exists('.git_credentials'):
    # This will read the API token from a local 
    github_username = "datalabNU"
    github_token = open('.git_credentials', 'r').read()
else:
    github_username = input("Please enter your GitHub username: ")
    github_token = getpass.getpass("Please enter your GitHub access token (register a token at github.com/settings/tokens): ")

# Log into GitHub
g = Github(github_token)
user = g.get_user()

In [27]:
user

AuthenticatedUser(login=None)

In [29]:
print(f"{github_username}/{repository_name}")

millicentli/prolific-deployment-ml


In [22]:
try:
    remote_repo = g.get_repo(f"{github_username}/{repository_name}").clone_url
    print(f"Found existing repository: {remote_repo}")
except Exception as e:
    remote_repo = user.create_repo(repository_name).clone_url
    print(f"Created new repository: {remote_repo}")

if not os.path.exists(repository_name):
    git.Repo.clone_from(remote_repo, repository_name)

repo = git.Repo(repository_name)


UnknownObjectException: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#create-a-repository-for-the-authenticated-user"}

In [16]:
for file_path in [local_template_path, local_data_path]:
    template_repo_path = os.path.join(os.getcwd(), repository_name, file_path)
    shutil.copy(file_path, template_repo_path)
    repo.index.add([template_repo_path])
repo.index.commit("Add a new interface template and data")

origin = repo.remote("origin")

In [17]:
origin.push()
print(f"Pushed changes to {remote_repo}")

Pushed changes to https://github.com/datalabNU/prolific-deployment.git


In [18]:
template_path = f"{github_username}/{repository_name}/master/{local_template_path}"
data_path = f"{github_username}/{repository_name}/master/{local_data_path}"

print(f"Hosted template: {template_path}")
print(f"Hosted data: {data_path}")

Hosted template: datalabNU/prolific-deployment/master/interface.yml
Hosted data: datalabNU/prolific-deployment/master/quarel.json


In [19]:
study_url = f"https://thresh.tools/?gh={template_path}"
study_url = study_url + "&PROLIFIC_PID={%PROLIFIC_PID%}&STUDY_ID={%STUDY_ID%}&SESSION_ID={%SESSION_ID%}"

In [20]:
print(f"View your hosted template here:\t\t  {study_url}")

study_url = f"{study_url}&d={data_path}"

print(f"Your study (with data) will be linked to: {study_url}")

View your hosted template here:		  https://thresh.tools/?gh=datalabNU/prolific-deployment/master/interface.yml&PROLIFIC_PID={%PROLIFIC_PID%}&STUDY_ID={%STUDY_ID%}&SESSION_ID={%SESSION_ID%}
Your study (with data) will be linked to: https://thresh.tools/?gh=datalabNU/prolific-deployment/master/interface.yml&PROLIFIC_PID={%PROLIFIC_PID%}&STUDY_ID={%STUDY_ID%}&SESSION_ID={%SESSION_ID%}&d=datalabNU/prolific-deployment/master/quarel.json
